In [20]:
from pyarrow import flight
import pyarrow as pa

import base64, time, os

import getpass

import pandas as pd

In [21]:
user = "uid98421"

password = getpass.getpass(prompt='Password: ')

host = 'dremio.aws1630.vitesco.io'

Password:  ········


In [22]:
sql = 'SELECT * FROM "rworkspace-space"."sales with bom costs" limit 10'

## Query Data with Arrow-Flight

In [23]:
class HttpDremioClientAuthHandler(flight.ClientAuthHandler):

    def __init__(self, username, password):
        super(flight.ClientAuthHandler, self).__init__()
        self.basic_auth = flight.BasicAuth(username, password)
        self.token = None

    def authenticate(self, outgoing, incoming):
        auth = self.basic_auth.serialize()
        outgoing.write(auth)
        self.token = incoming.read()

    def get_token(self):
        return self.token

In [24]:
def flight_query(sql,host):
    start = time.time()
    client = flight.FlightClient(f'grpc+tcp://{host}:32010')
    client.authenticate(HttpDremioClientAuthHandler(user, password)) 
    info = client.get_flight_info(flight.FlightDescriptor.for_command(sql + ' -- arrow flight'))
    reader = client.do_get(info.endpoints[0].ticket)
    batches = []
    while True:
        try:
            batch, metadata = reader.read_chunk()
            batches.append(batch)
        except StopIteration:
            break
    data = pa.Table.from_batches(batches)
    df = data.to_pandas()
    size = df.shape[0]
    elapsed = time.time() - start
    return df, size, elapsed

In [25]:
e_flight, s, df = flight_query(sql,host)
print(e_flight, s)

     fy  period recordtype customer_group material_type  product_hierarchy  \
0  2019       1          B            HMG          FERT  00F03DA150241PV13   
1  2019       1          B            HMG          FERT  00F03DA150241PV16   
2  2019       1          B            HMG          FERT  00F03DA150242PV13   
3  2019       1          B            HMG          FERT  00F03DA150245PV17   
4  2019       1          B            HMG          FERT  00F03DA150251PV12   
5  2019       1          B            HMG          FERT  00F03DA150251PV14   
6  2019       1          B            HMG          FERT  00F03DA150251PV15   
7  2019       1          B            HMG          FERT  00F03DA150251PV16   
8  2019       1          B            HMG          FERT  00F03DA150341OV01   
9  2019       1          B            HMG          FERT  00F03DA150341OV02   

                                PH_description   customer_engines  \
0          SIM2K-241 PCU LIN VCM ISG Smart-BEM         누우 MPI LIN   
1  